In [ ]:
# 必要なモジュールのインポート
import sys
import os

# プロジェクトのルートディレクトリをパスに追加
# Jupyter環境でのモジュール解決のため
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

from agent_tools import search_rag_knowledge_base, list_rag_collections, QdrantConnectionError, CollectionNotFoundError
from config import AgentConfig
from qdrant_client import QdrantClient
from qdrant_client.http.exceptions import UnexpectedResponse

# Qdrantクライアントの初期化（agent_tools内部でも行われているが、ここでも明示的に）
qdrant_url = "http://localhost:6333"
client = None
try:
    client = QdrantClient(url=qdrant_url)
    client.get_collections() # 接続テスト
    print("Qdrantサーバーに接続できました。")
except Exception as e:
    print(f"警告: Qdrantサーバーに接続できませんでした。サンプルコードを実行する前にQdrantサーバーを起動してください。")
    print(f"エラー: {e}")
    print("Qdrantサーバーは以下コマンドで起動できます: ")
    print("docker-compose -f docker-compose/docker-compose.yml up -d")
    # ここで処理を中断することもできますが、サンプルとして続行（ただし以降の検索は失敗します）
    # raise

print("\n--- 利用可能なコレクション一覧 ---")
test_collection_name = None
try:
    collections_info_str = list_rag_collections()
    print(collections_info_str)
    
    # 既存のコレクションから適切なものを選択するヒント
    # Qdrantを事前に登録している必要があるため、ここでは一般的なコレクション名を試します
    # もし特定のコレクション名を使用したい場合は、以下を変更してください。
    available_collections_list = []
    if "利用可能なコレクション一覧:" in collections_info_str:
        for line in collections_info_str.split('\n'):
            if line.startswith('- '):
                col_name = line.split(' ')[1] # '- col_name (count documents)'
                available_collections_list.append(col_name)

    # Gemini関連のコレクションを優先して選択
    for col in ["qa_corpus_gemini", "qa_cc_news_gemini", "qa_livedoor_gemini"]:
        if col in available_collections_list:
            test_collection_name = col
            break

    if test_collection_name is None:
        # Gemini関連が見つからない場合、AgentConfigのデフォルトを試す
        if AgentConfig.RAG_DEFAULT_COLLECTION in available_collections_list:
            test_collection_name = AgentConfig.RAG_DEFAULT_COLLECTION
        elif available_collections_list: # 何かコレクションがあれば最初のものを使う
            test_collection_name = available_collections_list[0]
        else:
            # どのコレクションも存在しない場合は、ユーザーにコレクション登録を促す
            test_collection_name = "your_collection_name_here" # 仮の名前
            print("\n警告: 利用可能なコレクションが見つかりません。")
            print(f"サンプルを実行するには、まずQdrantにデータを登録してください。（例: python a42_qdrant_gemini_registration.py --collection qa_corpus_gemini --path OUTPUT/preprocessed_wikipedia_ja_YYYYMMDD_HHMMSS.csv）")
            print(f"または、`test_collection_name` に存在するコレクション名を直接指定してください。")

except (QdrantConnectionError, UnexpectedResponse) as e:
    print(f"コレクション一覧の取得中にエラーが発生しました: {e}")
    test_collection_name = "qa_corpus_gemini" # エラー時フォールバック
    print(f"デフォルトのコレクション名 '{test_collection_name}' を使用して続行を試みますが、検索は失敗する可能性があります。")
except Exception as e:
    print(f"予期せぬエラー: {e}")
    test_collection_name = "qa_corpus_gemini" # エラー時フォールバック
    print(f"デフォルトのコレクション名 '{test_collection_name}' を使用して続行を試みますが、検索は失敗する可能性があります。")


# 適切なINPUT (クエリとコレクション名) を用意
test_query = "Geminiモデルの最新機能は何ですか？"
# test_collection_name は上記で取得した、またはデフォルトのコレクション名を使用


print(f"\n--- agent_tools.search_rag_knowledge_base 関数のテスト ---")
print(f"検索クエリ: \"{test_query}\"")
print(f"検索対象コレクション: \"{test_collection_name}\"")

if test_collection_name == "your_collection_name_here":
    print("**注意:** コレクション名が設定されていません。上記の指示に従ってコレクションを登録または名前を修正してください。")
elif client is None: # Qdrant接続エラーの場合
    print("**注意:** Qdrantサーバーに接続できていないため、検索は実行されません。サーバーを起動してから再度実行してください。")
else:
    try:
        # 関数を呼び出す
        result = search_rag_knowledge_base(query=test_query, collection_name=test_collection_name)

        # 期待するOUTPUTを得て表示
        print("\n--- 検索結果 ---")
        print(result)

        if "[[NO_RAG_RESULT]]" in result:
            print("\n**注意:** 検索結果が見つからなかった、またはスコアが閾値未満でした。")
            print("         Qdrantに適切なデータが登録されているか確認してください。")
        elif "[[RAG_TOOL_ERROR]]" in result:
            print("\n**エラー:** 検索ツール実行中にエラーが発生しました。詳細は上記ログを確認してください。")

    except Exception as e:
        print(f"\n予期せぬエラーが発生しました: {e}")
        print("Qdrantサーバーの起動状態、コレクションの存在、およびログファイルを確認してください。")
